# Scheduling

* 일정계획은 주어진 작업을 수행하는데 필요한 자원을 할당하는 활동
* 일정계획은 어떤 목적을 지향
	* 납기를 만족시킬 수 있기를 원할 수도 있고,
	* 단위시간당 생산량을 최대로 하기를 원할 수도 있고,
	* 기계 가동율을 높이는 것을 목적일 수도 있고,
	* 문제가 매우 복잡하고 제약조건이 많고 까다롭기 때문에 가능해(feasible solution)을 찾는 것 자체가 목적이 되기도 함
* 따라서 최적해를 구하기 위해서 모든 공간을 다 탐색하는 것은 실용적이지 않으며, 모형에 근거한 이론적인 분석을 통해 개발된 효율적 이론(theorem), 규칙(rule), 알고리즘(algorithm) 등을 사용하는 것이 현명함(Conway et al., 1967; Baker, 1974; French, 1982).

* 해석을 간단하게 하기 위해 대부분 정적(static)인 경우를 가정하는데, M개의 독립적인 작업이 시간 0에서 가공이 가능하다는 것이다. 만약 이러한 작업들이 임의로 작업장에 들어온다면 이것은 동적(dynamic)이한다.
	* 한 대의 기계는 지속적으로 사용가능하다.
	* 각 가공은 오직 한 대의 기계에서만 수행된다.
	* 각 기계는 한 번에 하나의 작업만 처리할 수 있다.

* 대량생산은 대개 assembly line system에서 이루어지고
* 단품생산은 single-site production system(항공기 제작업체나 조선소)
* job shop(금형, 공작기계 생산업체)
* FMS 등에서 많이 이루어진다는 것이다.
* 어떤 제조업체는 수주생산(make-to-order)과 계획생산(make-to-stock) 전략을 동시에 취할 수 있는 것처럼 제조설비의 layout이나 공정의 흐름은 여러 형태가 섞여 있을 수 있다는 것이다. 반도체나 금형 생산고정은 크게 흐름생산(flow production)을 따르고 있지만 부분적으로 job-shop형태의 설비배치나 작업흐름이 동시에 존재할수도 있음


### 스케줄링 분류
* 스케줄링(이하 일정계획) 문제는 매우 다양하기 때문에 분류가 쉽지 않지만, 당면한 문제를 풀기 위해 최소한 우리가 접한 문제의 유형을 파악할 수 있어야 기존의 연구결과를 활용하거나 개량하는 등의 접근이 가능

* 일정계획 문제를 구성하는 핵심은 자원, 작업, 목적으로 구분

* 대부분 일정계획 교재에서는, 
	* 단일기계 일정계획 문제(single machine scheduling problem)
	* 병렬기계 일정계획 문제(parallel machine scheduling problem)
	* 흐름공정 일정계획 문제(flow shop scheduling problem)
	* 개별공정 일정계획 문제(job-shop scheduling problem)
	* 프로젝트 일정계획 문제(project scheduling problem)

* 이러한 구분은 설비의 구성 및 배치, 작업의 흐름에 따른 것, 최근(학계) 이런 일반적인 구분보다 자세한 분류를 선호하는 경향이 발생



### 문제의 핵심으로 분류
* 작업(jobs)
	* 우선권(Preemption) 허용 여부:
		* 작업 $J_1$을 기계에서 가공 중인데, 그 작업 외에 긴급한 작업  $J_2$이 있을 경우, _𝐽_1_이 종료되기 전에 작업을 중지하고 긴급한 작업 _𝐽_2_을 먼저 처리할 것인지에 대한 여부
준비비용(setup cost)가 큰 경우와 기술적인 문제가 있는 경우 우선권을 허용하는가?

선행관계(precedence relation):
라우팅(routing)은 공정의 진행 순서를 나타내는데 이런 순서를 선행관계라고 하며, 예를 들어 면삭을 한 후에 드릴링을 한다고 하면 면삭공정이 드릴링공정의 선행공정이며 반드시 면삭공정이 끝난 후 드릴링공정이 가능
선행관계가 없다면 공정의 순서와 상관없이 작업이 가능

시작시기(release data, _𝑟_𝑗_):
작업이 시스템에 도착하는 것 즉, 작업을  시작할 수 있는 가장 빠른 시점
시작시기를 조정함으로써 생산시스템의 운용효율을 조정 가능


### One machine scheduling with release dates and deadlines

In [1]:
from pulp import *

prob = LpProblem('Minimize makespan', LpMinimize)

S1 = LpVariable('S1', lowBound=0, cat='Integer')
S2 = LpVariable('S2', lowBound=0, cat='Integer')
X12 = LpVariable('X11', lowBound=0, upBound=1, cat='Integer')

prob += S1 + S2 + 3 + 2

prob += S1 >= 0
prob += S2 >= 1
prob += S1 + 3 <= 9
prob += S1 + 2 <= 7
prob += S2 - S1 >= 3 - 100*(1-X12)
prob += S1 - S2 >= 2 - 100*X12

In [2]:
prob.solve()

print('Status: ', LpStatus[prob.status])

for v in prob.variables():
    print(v.name, '=', v.varValue)

Status:  Optimal
S1 = 0.0
S2 = 3.0
X11 = 1.0


### Single Machine Scheduling with Weighted Completion Times

In [6]:
from pulp import *

prob = LpProblem('Single Machine Scheduling with Weighted Completion Times', LpMinimize)

In [7]:
jobs = 5
p = [5, 6, 9, 12, 7]
w = [2, 3, 1, 9, 5]

In [8]:
# Define decision variables
x = LpVariable.dicts('x', [(j, k) for j in range(jobs) for k in range(jobs)], 
                     lowBound=0, upBound=1, cat='Integer')

# Define objective function
prob += lpSum([w[j] * p[k] * x[(k, j)] for j in range(jobs) for k in range(jobs)])

# Define constraints
# Either job j precedes job k or the other way around
for j in range(jobs):
    for k in range(jobs):
        if j != k:
            prob += x[(k, j)] + x[(j, k)] == 1

for j in range(jobs):
    for k in range(jobs):
        for l in range(jobs):
            if j != k and j != l and k != l:
                prob += x[(k, j)] + x[l, k] + x[(j, l)] >= 1

for j in range(jobs):
    prob += x[(j, j)] == 0

In [9]:
prob.solve(GUROBI())

print('Status', LpStatus[prob.status])
print(value(prob.objective))
for v in prob.variables():
    print(v.name, '=', v.varValue)

Optimize a model with 85 rows, 25 columns and 225 nonzeros
Variable types: 0 continuous, 25 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+00, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 363.0000000
Presolve removed 65 rows and 15 columns
Presolve time: 0.01s
Presolved: 20 rows, 10 columns, 60 nonzeros
Variable types: 0 continuous, 10 integer (10 binary)

Root relaxation: objective 1.970000e+02, 0 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     197.0000000  197.00000  0.00%     -    0s

Explored 0 nodes (0 simplex iterations) in 0.03 seconds
Thread count was 8 (of 8 available processors)

Solution count 2: 197 363 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.970000000000e+02, best bound 1.97